# 07B. Reaction-diffusion systems

### Mingyang Lu

### 3/7/2024

# Reaction-diffusion equationsIf we now use $u(x,t)$ as the state variable and add another term $f(u,t)$ to the diffusion equation, $$\frac{\partial u(X,t)}{\partial t} = f(u,t) + D \frac{\partial^2 u(X,t)}{\partial X^2} \tag{1} $$Equation (1) is called reaction-diffusion equation. $f(u, t)$ represents the reaction term, which we commonly used in a chemical rate equation$$\frac{du}{dt} = f(u, t) \tag{2}$$When $f(u, t)$ vanishes, Equation (1) becomes the PDE for a pure diffusion process. # Fisher's equationIf we consider a logistic growth as the reaction, Equation (2) for $u(X, t)$ becomes $$\frac{\partial u}{\partial t} = ru(1-\frac{u}{B}) + D \frac{\partial^2 u}{\partial X^2} \tag{3} $$$u$ is the population size, and $B$ is the carrying capacity. This is Fisher's equation, first proposed by Ronald Fisher in 1934. Below is a slightly modified PDE integrator to model reaction-diffusion systems. We use the Dirichlet boundary condition here (zero $u$ at the boundary).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## PDE integration with the finite difference method for reaction-diffusion equation
def pde_fd_reaction_diffusion(derivs, ngrid, dX, dt, D, t0, t_total, p0, *args):
    """
    Parameters:
    - derivs: derivative function
    - ngrid: number of grid points
    - dX: X step size
    - dt: time step size
    - D: diffusion constant
    - t0: initial time
    - t_total: total simulation time
    - p0: initial condition: P(X, t = t0)
    - *args: additional arguments to the derivative function
    
    Returns:
    - p: Final values of P(X, t)
    """
    t_all = np.arange(t0, t_total + dt, dt)
    nt_all = len(t_all)
    factor = D / dX**2 * dt
    p = p0.copy()  # Make a copy to avoid modifying the original array

    for i in range(nt_all - 1):
        p_plus_one = np.concatenate((p[1:], [0]))  # P(X + dX) for all Xs, 0 for boundary condition
        p_minus_one = np.concatenate(([0], p[:-1]))  # P(X - dX) for all Xs, 0 for boundary condition
        
        # Calculate derivatives at each point
        f = np.array([derivs(t_all[i], X, *args) for X in p])
        
        # Update P(X, t) using finite difference method
        p = p + f * dt + factor * (p_plus_one + p_minus_one - 2 * p)

    return p